In [1]:
!pip install yfinance
!pip install pandas
!pip install requests
!pip install bs4
!pip install plotly
!pip install nbformat --upgrade

In [2]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

# Question 1: Use yfinance to Extract Stock Data

In [4]:
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period = "max")
tesla_data.reset_index(inplace = True)
tesla_data.head(5)


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


# Question 2: Use Webscraping to Extract Tesla Revenue Data


In [5]:
# Send an HTTP request to the URL of the webpage you want to access
url = 'https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue'
res = requests.get(url)

# Create a BeautifulSoup object to parse the HTML content
soup = BeautifulSoup(res.content,'html.parser')

# Find the revenue table body by inspecting the HTML code
revenue_table_body = soup.find_all("tbody")[1]

# Initialize an empty list to hold the row dictionaries
rows = []

# Iterate through each row of the revenue table body and extract the date and revenue values
for row in revenue_table_body.find_all("tr"):
    col = row.find_all("td")
    date = col[0].text
    revenue = col[1].text.replace("$", "").replace(",", "")
    
    # Append the date and revenue values to the rows list as a dictionary
    rows.append({"Date": date, "Revenue": revenue})

# Create a new pandas DataFrame by passing the rows list to the pd.DataFrame constructor
tesla_revenue = pd.DataFrame(rows)

# Clean the 'Revenue' column by removing commas and dollar signs, and dropping any rows with missing values
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")
tesla_revenue.dropna(inplace=True)

# Remove any rows with empty values in the 'Revenue' column
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

print(tesla_revenue.tail(5))

          Date Revenue
49  2010-09-30      31
50  2010-06-30      28
51  2010-03-31      21
53  2009-09-30      46
54  2009-06-30      27


# Question 3: Use yfinance to Extract Stock Data


In [6]:
gme = yf.Ticker("GME")
gme_data = gme.history(period = "max")

gme_data.reset_index(inplace=True)
gme_data.head(5)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716073,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615921,1.662210,1.603296,1.662210,6892800,0.0,0.0


# Question 4: Use Webscraping to Extract GME Revenue Data


In [7]:
# Send an HTTP request to the URL of the webpage you want to access
url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
res = requests.get(url)

# Create a BeautifulSoup object to parse the HTML content
soup = BeautifulSoup(res.content,'html.parser')
# Find the revenue table body by inspecting the HTML code
revenue_table_body = soup.find_all("tbody")[1]
# Initialize an empty list to hold the row dictionaries
rows = []
# Iterate through each row of the revenue table body and extract the date and revenue values
for row in revenue_table_body.find_all("tr"):
    col = row.find_all("td")
    date = col[0].text
    revenue = col[1].text.replace("$", "").replace(",", "")
    
    # Append the date and revenue values to the rows list as a dictionary
    rows.append({"Date": date, "Revenue": revenue})

# Create a new pandas DataFrame by passing the rows list to the pd.DataFrame constructor
gme_revenue = pd.DataFrame(rows)

# Clean the 'Revenue' column by removing commas and dollar signs, and dropping any rows with missing values
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$',"")
gme_revenue.dropna(inplace=True)

# Remove any rows with empty values in the 'Revenue' column
gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

# Display the last five rows of the gme_revenue DataFrame
print(gme_revenue.tail(5))

          Date Revenue
52  2010-01-31    3524
53  2009-10-31    1835
54  2009-07-31    1739
55  2009-04-30    1981
56  2009-01-31    3492


# Question 5: Plot Tesla Stock Graph


In [8]:
make_graph(tesla_data, tesla_revenue, 'Tesla')


C:\Users\Administrator\AppData\Local\Temp\ipykernel_3208\2068038883.py:5: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\Administrator\AppData\Local\Temp\ipykernel_3208\2068038883.py:6: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



# Question 6: Plot GameStop Stock Graph


In [9]:
make_graph(gme_data, gme_revenue, 'GameStop')


C:\Users\Administrator\AppData\Local\Temp\ipykernel_3208\2068038883.py:5: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\Administrator\AppData\Local\Temp\ipykernel_3208\2068038883.py:6: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

